# Sentence Transformer ONNX model export

This notebook demonstrates export of a sentence-transformers model to ONNX format and also quantization of the model

See https://huggingface.co/sentence-transformers/msmarco-MiniLM-L-6-v3
    

In [ ]:
!pip install transformers torch onnx onnxruntime 

In [ ]:
from transformers import AutoModel, AutoTokenizer, BertTokenizer, BertPreTrainedModel, BertModel
import transformers
import torch 
from pathlib import Path

We create a wrapper model so that we can compute the mean pooling over the output inside ONNX. Almost all sentence-transformer models uses mean pooling. 


In [ ]:
class MeanPoolingEncoderONNX(BertPreTrainedModel):

    def __init__(self,config):
        super().__init__(config)
        self.bert = BertModel(config)
        self.init_weights()
        
    def forward(self, input_ids, attention_mask, token_type_ids=None):
        token_embeddings = self.bert(input_ids,attention_mask=attention_mask)[0]
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
        sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
        sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
        return sum_embeddings / sum_mask

In [ ]:
encoder = MeanPoolingEncoderONNX.from_pretrained("sentence-transformers/msmarco-MiniLM-L-6-v3")
tokenizer = BertTokenizer.from_pretrained("sentence-transformers/msmarco-MiniLM-L-6-v3")

In [ ]:
encoder = encoder.eval()

In [ ]:
pipeline = transformers.Pipeline(model=encoder, tokenizer=tokenizer)

In [ ]:
import transformers.convert_graph_to_onnx as onnx_convert
onnx_convert.convert_pytorch(pipeline, opset=11, output=Path("sentence-encoder.onnx"), use_external_format=False)

In [ ]:
onnx_convert.quantize(Path("sentence-encoder.onnx"))

Now we can test the model using ONNX Runtime 


In [ ]:
import onnxruntime as rt
import numpy

In [ ]:
session = rt.InferenceSession("sentence-encoder-quantized.onnx")

In [ ]:
session.get_outputs()[0].shape

In [ ]:
session.get_outputs()[0].name

In [ ]:
inputs = tokenizer("this is a test", return_tensors="np") #Tokenization outside of ONNX model..

In [ ]:
onnx_input_dict = {
    "input_ids": inputs['input_ids'],
    "attention_mask": inputs['attention_mask'],
}
embedding = session.run(input_feed=onnx_input_dict, output_names=['output_0'])[0][0]